In [1]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np
from tableone_modified.tableone import TableOne
import warnings
warnings.filterwarnings("ignore")
from utils import *

In [2]:
combined,var_list,cat_features_list,cat_order = get_data()
combined,var_list,cat_features_list,cat_order = apply_exclusions(combined, var_list,cat_features_list,cat_order)

Cohort Size: 289
No anticoagulation: 14

Breakdown of outcomes:
Resolved     156
Recurrent     14
Name: lvtrecurrence, dtype: int64
Died          53
Persistent    21
Name: statusofdeath, dtype: int64

Unknown outcome: 31
Final cohort size:244



In [3]:
mytable = TableOne(combined, var_list, categorical=cat_features_list, groupby='lvtstatus',order=cat_order,pval=True,remarks=False,reverse_missing=True)
HTML(mytable.tabulate(tablefmt="html"))

,,Count,Resolved LVT,Unresolved LVT/Death,P-Value,Test
n,,,156,88,,
"Age, years",,244,58.4 (12.1),61.5 (14.3),0.090,Two Sample T-test
Sex,Male,244,134 (85.9),78 (88.6),0.681,Chi-squared
,Female,,22 (14.1),10 (11.4),,
"Height, cm",,190,1.7 (0.1),1.6 (0.1),0.168,Two Sample T-test
"Weight, kg",,220,70.4 (17.5),66.0 (14.1),0.045,Two Sample T-test
Body Mass Index,,193,26.2 (6.0),25.0 (4.4),0.121,Two Sample T-test
Diabetes Mellitus/Prediabetes,No,244,83 (53.2),44 (50.0),0.728,Chi-squared
,Yes,,73 (46.8),44 (50.0),,
Chronic Kidney Disease,No,244,139 (89.1),68 (77.3),0.022,Chi-squared


In [4]:
mytable.to_excel('tableone.xlsx')

In [5]:
for status in ['All Cases','Resolved LVT','Unresolved LVT/Death']:
    if status == 'All Cases':
        series = combined['Followup Duration, days']
    else:
        series = combined['Followup Duration, days'][combined['lvtstatus'] == status]
    print(status)
    print(f'Median: {series.median()}')
    print(f'IQR: {series.quantile(0.25)}-{series.quantile(0.75)}')

All Cases
Median: 807.0
IQR: 264.75-1746.25
Resolved LVT
Median: 1049.0
IQR: 598.5-2163.5
Unresolved LVT/Death
Median: 196.5
IQR: 15.75-908.25


In [6]:
combined = combined.drop('Followup Duration, days',axis=1)
combined['lvtstatus'] = combined['lvtstatus'].replace({'Resolved LVT':1,'Unresolved LVT/Death':0})
combined['lvtstatus'].to_csv('processed_data/outcome.csv',index=False)
predictors = combined.drop('lvtstatus',axis=1)
predictors.to_csv('processed_data/predictors.csv',index=False)
pd.Series(cat_features_list).to_csv('processed_data/categorical_features.csv',index=False)
pd.Series(list(set(list(predictors)) - set(cat_features_list))).to_csv('processed_data/numeric_features.csv',index=False)